In [11]:
%env XLA_PYTHON_CLIENT_PREALLOCATE = False
import os
import genjax
import numpy as np
import bayes3d as b
from genjax.generative_functions.distributions import ExactDensity
import jax.numpy as jnp
import bayes3d.genjax
import jax
from jax.debug import print as jprint
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML
import PIL

def display_video(frames, framerate=30):
    if type(frames[0]) == PIL.Image.Image:
      frames = [np.array(frames[i]) for i in range(len(frames))]
      print(frames[0].shape)
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=True)
    return HTML(anim.to_html5_video())

def video_from_trace(trace, scale = 8, use_retval = False):
    if use_retval:
        depths = trace.get_retval()[0]
    else:
        depths = trace["depths", "depths"]
    images = [b.scale_image(b.get_depth_image(depths[i,...,2]),scale) for i in range(depths.shape[0])]
    return images

console = genjax.pretty()

env: XLA_PYTHON_CLIENT_PREALLOCATE=False


In [3]:
b.setup_visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [4]:
intrinsics = b.Intrinsics(
    height=50,
    width=50,
    fx=250.0, fy=250.0,
    cx=25.0, cy=25.0,
    near=0.01, far=20.0
)

b.setup_renderer(intrinsics)
b.RENDERER.add_mesh_from_file(os.path.join(b.utils.get_assets_dir(), "sample_objs/cube.obj"), scaling_factor=0.1)

[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (64, 64, 1024)


In [5]:
T = 10

@genjax.gen
def dynamics_model_v1(prev):
    (t, pose, velocity) = prev
    velocity = b.gaussian_vmf_pose(velocity, 0.005, 10000.0)  @ f"velocity"
    pose = b.gaussian_vmf_pose(pose @ velocity, 0.005, 10000.0)  @ f"pose"
    return (t + 1, pose, velocity)

dynamics_model_v1_unfold = genjax.UnfoldCombinator.new(dynamics_model_v1, T+1)
image_likelihood_vmap = genjax.MapCombinator.new(genjax.gen(lambda *x: b.image_likelihood(*x) @ "depths"), in_axes=(0,None,None,None,None))

In [6]:
@genjax.gen
def model_single_object(T_vec, outlier_volume, focal_length):
    T = T_vec.shape[0]
    pose = b.uniform_pose(jnp.array([-0.1,-0.1,1.5]), jnp.array([0.1,0.1,2])) @ "init_pose"
    velocity = b.gaussian_vmf_pose(jnp.eye(4), 0.01, 10000.0) @ "init_velocity"
    dynamics = dynamics_model_v1_unfold(T,(0, pose, velocity)) @ "dynamics"
    poses = jax.lax.slice_in_dim(dynamics[1], 0, T+1, axis = 0) 

    # poses = jax.lax.cond(poses.shape[0] < 4, lambda x: poses[:,None,...], lambda x: poses, (None,))
    indices = jnp.array([0])

    rendered_images = b.RENDERER.render_many(
        poses[:,None,...], indices 
    )[...,:3]

    variance = genjax.distributions.tfp_uniform(0.00000000001, 10000.0) @ "variance"
    outlier_prob  = genjax.distributions.tfp_uniform(-0.01, 10000.0) @ "outlier_prob"
    
    # images = b.image_likelihood(rendered_images[0,...], variance, outlier_prob, outlier_volume, focal_length) @ "images"
    images = image_likelihood_vmap(rendered_images, variance, outlier_prob, outlier_volume, focal_length) @ "depths"
    return rendered_images, poses


model_single_object_simulate_jit = jax.jit(model_single_object.simulate)
model_single_object_importance_jit = jax.jit(model_single_object.importance)


In [7]:
# key = jax.random.PRNGKey(31415)
# gt_tr = model_single_object_simulate_jit(key, (jnp.zeros(T), jnp.float64(1000.0), jnp.float64(1.0)))
# poses = tr["dynamics"]["pose"]
# for i in range(poses.shape[0]):
#     b.show_pose(f"{i}", poses[i])
# display_video(video_from_trace(gt_tr, 0.7),  framerate=24)

# def c2f_pose_update(trace_, key,  number, contact_param_deltas, VARIANCE_GRID, OUTLIER_GRID):
#     contact_param_grid = contact_param_deltas + trace_[f"contact_params_{number}"]
#     scores = contact_enumerators[number][3](trace_, key, contact_param_grid, VARIANCE_GRID, OUTLIER_GRID)
#     i,j,k = jnp.unravel_index(scores.argmax(), scores.shape)
#     return contact_enumerators[number][0](
#         trace_, key,
#         contact_param_grid[i], VARIANCE_GRID[j], OUTLIER_GRID[k]
#     )

In [8]:
key = jax.random.PRNGKey(31415)
subkey = jax.random.split(key, 1)[0]
weight, gt_trace = model_single_object_importance_jit(subkey, genjax.choice_map({
    "variance": 0.0001,
    "outlier_prob": 0.0001,
}), (jnp.zeros(T), jnp.float32(1000.0), jnp.float32(1.0)))

In [14]:
display_video(video_from_trace(gt_trace, 4, use_retval = True),  framerate=24)

(200, 200, 3)


/tmp/ipykernel_71886/4150747562.py:26: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.
  matplotlib.use(orig_backend)  # Switch back to the original backend.


<IPython.core.display.HTML object>

In [8]:
gt_trace.get_score()

Array(89422.77, dtype=float32)

In [13]:
subkey = jax.random.split(subkey, 5446)[0]
subkey = jax.random.PRNGKey(5436456)
chm = genjax.choice_map(
    {"depths" : genjax.vector_choice_map(
        genjax.choice_map({
            "depths":gt_trace.get_choices()["depths", "depths"]
        })
    ),
    "variance": 0.0001,
    "outlier_prob": 0.0001}
)
# chm = genjax.vector_choice_map(genjax.choice_map({("depths", "depths"):gt_trace.get_choices()["depths", "depths"]}))
w,trace = model_single_object_importance_jit(subkey, chm, (jnp.zeros(T), jnp.float32(1000.0), jnp.float32(1.0)))
w

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 │   "outlier_prob": 0.0001}                                                                 │
│   11 )                                                                                           │
│   12 # chm = genjax.vector_choice_map(genjax.choice_map({("depths", "depths"):gt_trace.get_ch    │
│ ❱ 13 w,trace = model_single_object_importance_jit(subkey, chm, (jnp.zeros(T), jnp.float32(100    │
│   14 w                                                                                           │
│   15                                                                                             │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/traceback_util.py:177 in            │
│ reraise_with_filtered_traceback                                                                  │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/pjit.py:256 in cache_miss           │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/pjit.py:167 in _python_pjit_helper  │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/core.py:2656 in bind                │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/core.py:388 in bind_with_trace      │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/core.py:868 in process_primitive    │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/pjit.py:1212 in _pjit_call_impl     │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/pjit.py:1196 in                     │
│ call_impl_cache_miss                                                                             │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/pjit.py:1129 in                     │
│ _pjit_call_impl_python                                                                           │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/pjit.py:1260 in _pjit_lower         │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/pjit.py:1299 in _pjit_lower_cached  │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/profiler.py:340 in wrapper          │
│                                                                                                  │
│ /opt/conda/envs/bayes3d/lib/python3.9/site-packages/jax/_src/interpreters/pxla.py:2031 in        │
│ lower_sharding_computation                                                                       │
│                                                            

In [12]:
display_video(video_from_trace(trace, 4, use_retval = True),  framerate=24)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 display_video(video_from_trace(trace, 4, use_retval = True),  framerate=24)                  │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'trace' is not defined

In [12]:
# viz_images = []
# max_depth = 10.0

# # inferred_poses_with_occ = jnp.stack([inferred_poses, occ1_poses], axis = 1)
# occ_image = b.viz.get_depth_image(b.RENDERER.render(occ1_pose[None,...], jnp.array([1]))[:,:,2])

# pred_images = b.RENDERER.render_many(inferred_poses[:,None, ...], jnp.array([0]))

# pred_with_occ_images = [b.overlay_image(b.viz.get_depth_image(pred_images[i,:,:,2]), 
# occ_image, alpha=0.4) for i in range(pred_images.shape[0])]

# gt_images = b.RENDERER.render_many(gt_poses[:,None, ...], jnp.array([0]))
# gt_with_occ_images = [b.overlay_image(b.viz.get_depth_image(gt_images[i,:,:,2]), 
# occ_image, alpha=0.5) for i in range(pred_images.shape[0])]

# viz_images = [
#     b.viz.multi_panel(
#         [g, b.viz.get_depth_image(p[:,:,2]), po],
#         labels = ["Ground Truth", "Reconstruction w/o Occluder", "Reconstruction w Occluder"],
#         title = "Scene 11A",
#         # bottom_text = "3DP3 + Physics Prior v1"
#     )
#     for (g, p, po) in zip(gt_with_occ_images, pred_images, pred_with_occ_images)
# ]
# display_video(viz_images)

In [11]:
imp = genjax.inference.importance_sampling.sampling_importance_resampling(model_single_object, 100)
(tr, lnw, log_ml_estimate) = imp.apply(subkey, chm, (jnp.zeros(T), jnp.float32(1000.0), jnp.float32(1.0)))

In [12]:
display_video(video_from_trace(tr, 4, use_retval = True),  framerate=24)

/tmp/ipykernel_2637/736626789.py:25: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.
  matplotlib.use(orig_backend)  # Switch back to the original backend.


<IPython.core.display.HTML object>

In [13]:
tr.get_score()

Array(43208.86, dtype=float32)

In [14]:
T = 5
pose_bounds = jnp.array([[-0.1,-0.1,1.5],[0.1,0.1,2]])
key = jax.random.PRNGKey(31415)
weight, gt_trace = model_v1_importance_jit(key, genjax.choice_map({
    "variance": 0.0001,
    "outlier_prob": 0.0001,
    "indices" : jnp.array([23])
}), (jnp.zeros(T), jnp.arange(len(b.RENDERER.meshes)), jnp.zeros(1),
b.RENDERER.model_box_dims, pose_bounds, jnp.float32(1000.0), jnp.float32(1.0)))

Array(42709.805, dtype=float32)

In [ ]:
model_simulate_jit = eval("model_v{}_simulate_jit".format(metadata["model_version"]))
model_importance_jit = eval("model_v{}_importance_jit".format(metadata["model_version"]))